In [ ]:
import open3d as o3d

point_cloud = o3d.io.read_point_cloud("/home/ainhoaarnaiz/final.ply")
o3d.visualization.draw_geometries([point_cloud])

In [ ]:
import open3d as o3d
import numpy as np

# Cargar la nube de puntos directamente
pcd = o3d.io.read_point_cloud("/home/ainhoaarnaiz/final_v2.ply")

# Eliminar normales si hacen que se vea como una superficie
pcd.normals = o3d.utility.Vector3dVector([])

def custom_draw(vis):
    opt = vis.get_render_option()
    opt.point_size = 1.0
    opt.background_color = np.asarray([0, 0, 0])
    return False  # only run once

o3d.visualization.draw_geometries_with_animation_callback([pcd], custom_draw)

In [ ]:
import open3d as o3d
import numpy as np
import colorsys
from sklearn.cluster import KMeans

# Load point cloud
pcd = o3d.io.read_point_cloud("/home/ainhoaarnaiz/final_v2.ply")
colors_rgb = np.asarray(pcd.colors)

# Convert RGB to HSV
colors_hsv = np.array([colorsys.rgb_to_hsv(*rgb) for rgb in colors_rgb])
hues = colors_hsv[:, 0].reshape(-1, 1)

# Cluster into 3 dominant hue groups
kmeans = KMeans(n_clusters=6, random_state=42)
kmeans.fit(hues)
labels = kmeans.labels_
centers = kmeans.cluster_centers_

# Count and sort
unique, counts = np.unique(labels, return_counts=True)
dominant = sorted(zip(centers.flatten(), counts), key=lambda x: -x[1])

# Print top 3 hue ranges (in degrees)
for i, (hue, count) in enumerate(dominant):
    print(f"Hue Cluster {i+1}: {round(hue*360, 1)}°, Count: {count}")

In [ ]:
import open3d as o3d
import numpy as np
import colorsys

def rgb_to_hsv(rgb):
    return colorsys.rgb_to_hsv(*rgb)

def mark_as_red(hsv):
    h, s, v = hsv
    return (90/360 <= h <= 160/360 and s >= 0.0 and v >= 0.0)

def mark_as_green(hsv):
    h, s, v = hsv
    return (0/360 <= h <= 90/360 and s >= 0.0 and v <= 0.8)

# Load and process point cloud
pcd = o3d.io.read_point_cloud("/home/ainhoaarnaiz/final_v2.ply")
pcd.normals = o3d.utility.Vector3dVector([])

colors = np.asarray(pcd.colors)
new_colors = []
for rgb in colors:
    hsv = rgb_to_hsv(rgb)
    if mark_as_red(hsv):
        new_colors.append([1, 0, 0])
    elif mark_as_green(hsv):
        new_colors.append([0, 1, 0])
    else:
        new_colors.append([0, 0, 1])
pcd.colors = o3d.utility.Vector3dVector(np.array(new_colors))

def custom_draw(vis):
    opt = vis.get_render_option()
    opt.point_size = 1.0
    opt.background_color = np.asarray([0, 0, 0])
    return False  # only run once

o3d.visualization.draw_geometries_with_animation_callback([pcd], custom_draw)


In [ ]:
import open3d as o3d
import numpy as np
import colorsys
from collections import defaultdict, Counter
from tqdm import tqdm

def rgb_to_hsv(rgb):
    return colorsys.rgb_to_hsv(*rgb)

def mark_as_red(hsv):
    h, s, v = hsv
    return 90/360 <= h <= 160/360

def mark_as_green(hsv):
    h, s, v = hsv
    return 0/360 <= h <= 90/360 and v <= 0.8

# Load point cloud
pcd = o3d.io.read_point_cloud("/home/ainhoaarnaiz/final_v2.ply")
pcd.normals = o3d.utility.Vector3dVector([])

points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)
labels = []

# Classify each point
for rgb in colors:
    hsv = rgb_to_hsv(rgb)
    if mark_as_red(hsv):
        labels.append("red")
    elif mark_as_green(hsv):
        labels.append("green")
    else:
        labels.append("blue")

# Voxel clustering
voxel_size = 0.9 #(0.05->0.8)
voxel_index = np.floor(points / voxel_size).astype(np.int32)

# Map voxel index → list of point indices
voxel_dict = defaultdict(list)
for idx, voxel in enumerate(voxel_index):
    voxel_dict[tuple(voxel)].append(idx)

# Majority vote per voxel
new_colors = np.zeros_like(colors)
for voxel_key, indices in tqdm(voxel_dict.items(), desc="Voxel clustering"):
    group_labels = [labels[i] for i in indices]
    majority = Counter(group_labels).most_common(1)[0][0]
    color = {"red": [1, 0, 0], "green": [0, 1, 0], "blue": [0, 0, 1]}[majority]
    for i in indices:
        new_colors[i] = color

pcd.colors = o3d.utility.Vector3dVector(new_colors)

# Visualize
def custom_draw(vis):
    opt = vis.get_render_option()
    opt.point_size = 1.0
    opt.background_color = np.asarray([0, 0, 0])
    return False

o3d.visualization.draw_geometries_with_animation_callback([pcd], custom_draw)

In [ ]:
import open3d as o3d
import numpy as np
import colorsys
from collections import defaultdict, Counter
from tqdm import tqdm

def rgb_to_hsv(rgb):
    return colorsys.rgb_to_hsv(*rgb)

def mark_as_red(hsv):
    h, s, v = hsv
    return 90/360 <= h <= 160/360

def mark_as_green(hsv):
    h, s, v = hsv
    return 0/360 <= h <= 90/360 and v <= 0.8

# Load point cloud
pcd = o3d.io.read_point_cloud("/home/ainhoaarnaiz/final_v2.ply")
pcd.normals = o3d.utility.Vector3dVector([])

points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)
labels = []
top = 3

# Classify each point
for rgb in colors:
    hsv = rgb_to_hsv(rgb)
    if mark_as_red(hsv):
        labels.append("red")
    elif mark_as_green(hsv):
        labels.append("green")
    else:
        labels.append("blue")

# Voxel clustering
voxel_size = 0.4
voxel_index = np.floor(points / voxel_size).astype(np.int32)

# Map voxel index → list of point indices
voxel_dict = defaultdict(list)
for idx, voxel in enumerate(voxel_index):
    voxel_dict[tuple(voxel)].append(idx)

# Majority vote per voxel
voxel_labels = {}
for voxel_key, indices in voxel_dict.items():
    group_labels = [labels[i] for i in indices]
    majority = Counter(group_labels).most_common(1)[0][0]
    voxel_labels[voxel_key] = majority

# Determine greenest clusters
green_voxel_scores = {}
neighbor_offsets = [
    (-1, 0, 0), (1, 0, 0),
    (0, -1, 0), (0, 1, 0),
    (0, 0, -1), (0, 0, 1)
]

for voxel_key in voxel_dict:
    if voxel_labels[voxel_key] != "green":
        continue
    neighbor_greens = 0
    for offset in neighbor_offsets:
        neighbor_key = tuple(np.array(voxel_key) + np.array(offset))
        if voxel_labels.get(neighbor_key) == "green":
            neighbor_greens += 1
    green_voxel_scores[voxel_key] = neighbor_greens

# Get top 3 greenest clusters
top_green_clusters = sorted(green_voxel_scores.items(), key=lambda x: -x[1])[:top]
top_green_voxels = set([v[0] for v in top_green_clusters])

# Print center positions of the top 3 clusters -------------------> -0.200, 5.000, -2.200
print("Top greenest cluster centers (x, y, z):")
for voxel in top_green_voxels:
    center = (np.array(voxel) + 0.5) * voxel_size
    print(f"{center[0]:.3f}, {center[1]:.3f}, {center[2]:.3f}")

# Assign final colors
new_colors = np.zeros_like(colors)
for voxel_key, indices in voxel_dict.items():
    if voxel_key in top_green_voxels:
        for i in indices:
            new_colors[i] = [0, 1, 0]  # bright green
    else:
        for i in indices:
            new_colors[i] = [0.2, 0.2, 0.2]  # dark gray

pcd.colors = o3d.utility.Vector3dVector(new_colors)

# Visualize
def custom_draw(vis):
    opt = vis.get_render_option()
    opt.point_size = 1.0
    opt.background_color = np.asarray([0, 0, 0])
    return False

o3d.visualization.draw_geometries_with_animation_callback([pcd], custom_draw)
